## 6. Deploy LVA & ML Solution to IoT Edge Device to handle Multiple Cameras
Unless specified, all commands, cells below will be running on the development/deployment machine (not on the edge device)

### 6.1. Setting up IoT Edge deployment target

#### 6.1.1. Get stored variables & set new variables
We will read the previously stored variables

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

acrServiceName = get_key(envPath, "acrServiceName")
containerImageName = get_key(envPath, "containerImageName")
acrUserName = get_key(envPath, "acrUserName")
acrPassword = get_key(envPath, "acrPassword")
acrServer = get_key(envPath, "acrServer")
resourceGroupName = get_key(envPath, "resourceGroupName")
iotHubServiceName = get_key(envPath, "iotHubServiceName")
iotDeviceId = get_key(envPath, "iotDeviceId")
iotEdgeDeviceConnString = get_key(envPath, "iotEdgeDeviceConnString")
localacrServiceName = get_key(envPath, "localacrServiceName")


# Set deployment specific variables for current deployment
hostDebugFolder = "/tmp/aix_debug_cam"
lvaModuleAddress = "marketplace.azurecr.io/azure-media-services-preview/official/media_edge:1.3-linux-x64"

# Set to the scoring URI of the AI Extensibility module
# DONT USE http://127.0.0.1 or http://localhost but use http://<MODULE_NAME>:<PORT_NUMBER>/<SCORING_PATH>
# Scoring URI address MUST be same as the container name of the AI module!
scoringURI = "http://" + containerImageName + ":5001/score"

# Template Manifest filenames
templateManifestFileName = "LVADeploymentManifestTemplate.json"
deploymentManifestFileName = "LVADeploymentManifest.json"
resetDeviceManifestTemplate = "LVAResetDeviceManifest.json"

#### Set camera addresses and related parameters
Per IP camera you want to connect to IoT Edge device, set the following parameter list:

Template for **single** camera:  
```python
    lvaSettings = [
        ["<IP_CAM_1_RTSP_ENDPOINT>", "<IP_CAM_1_USERNAME>", "<IP_CAM_1_PASSWORD>", "CAM_1_MOTION_SENSITIVITY", "<CAM_1_FPS_FRAME_RATE_FILTER>"]
    ]
```
Sample for **single** camera:
```python
    lvaSettings = [
        ["rtsp://192.168.1.101", "user", "LVAD3m0!", "high", "2"]
    ]
```

Template for **multiple** cameras:
```python
    lvaSettings = [
        ["<IP_CAM_1_RTSP_ENDPOINT>", "<IP_CAM_1_USERNAME>", "<IP_CAM_1_PASSWORD>", "CAM_1_MOTION_SENSITIVITY", "<CAM_1_FPS_FRAME_RATE_FILTER>"],
        ["<IP_CAM_2_RTSP_ENDPOINT>", "<IP_CAM_2_USERNAME>", "<IP_CAM_2_PASSWORD>", "CAM_2_MOTION_SENSITIVITY", "<CAM_2_FPS_FRAME_RATE_FILTER>"],
        ...
        ["<IP_CAM_n_RTSP_ENDPOINT>", "<IP_CAM_n_USERNAME>", "<IP_CAM_n_PASSWORD>", "CAM_n_MOTION_SENSITIVITY", "<CAM_n_FPS_FRAME_RATE_FILTER>"]
    ]
```
Sample for **multiple** camera:
```python
    lvaSettings = [
        ["rtsp://192.168.1.101", "user", "LVAD3m0!", "high", "2"],
        ["rtsp://192.168.1.102", "user", "LVAD3m0!", "low", "1"],
        ["rtsp://192.168.1.103", "user", "LVAD3m0!", "high", "1"],
        ["rtsp://192.168.1.104", "user", "LVAD3m0!", "medium", "2"],
        ["rtsp://192.168.1.105", "user", "LVAD3m0!", "high", "2"],
        ["rtsp://192.168.1.106", "user", "LVAD3m0!", "low", "5"]
    ]
```




In [ ]:
# LVA camera settings
lvaSettings = [
    ["rtsp://192.168.1.101", "user", "LVAD3m0!", "high", "2"]
]

#### 6.1.2. Set ACR or LocalRepo flag

In the previous section, we deploy the compiled docker image into Local Docker Repository or into ACR. Depending on where you deploy, set the below cell accordingly. If you are using local docker repository, than following cell should be  
```python
useACRFlag = False
```

if you are using ACR, than it should be  
```python
useACRFlag = True
```

In [ ]:
useACRFlag = False

#### 6.1.3. Create a manifest file from the template
Create a new manifest file from the template. Later we will update it to reflect the new settings

In [ ]:
from shutil import copyfile

# Create a copy from the template
copyfile(templateManifestFileName, deploymentManifestFileName)

#### 6.1.4. Update manifest file with AI module settings


In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

aiModule = deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<AIX_MODULE>"]
del deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<AIX_MODULE>"]

aiModuleS = json.dumps(aiModule)
if useACRFlag:
    aiModuleS = aiModuleS.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', acrServiceName+".azurecr.io")
else:
    aiModuleS = aiModuleS.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', localacrServiceName)
aiModuleS = aiModuleS.replace('AIX_CONTAINER_MODULE_NAME', containerImageName)
aiModuleS = aiModuleS.replace('AIX_HOST_DEBUG_FOLDER', hostDebugFolder)

aiModule = json.loads(aiModuleS)

deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"][containerImageName] = aiModule

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

#### 6.1.5. Update manifest file with LVA module settings

In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

lvaModule = deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<LVA_MODULE>"]
del deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["<LVA_MODULE>"]

lvaModuleS = json.dumps(lvaModule)
lvaModuleS = lvaModuleS.replace('AIX_LVA_MODULE_ADDRESS', lvaModuleAddress)

lvaModule = json.loads(lvaModuleS)

for s in lvaSettings:
    deploymentManifest["modulesContent"]["$edgeAgent"]["properties.desired"]["modules"]["lva" + s[0][-3:]] = lvaModule

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

#### 6.1.6. Set module properties in the deployment manifest file


In [ ]:
import json

with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

aiModuleProp = deploymentManifest["modulesContent"]["<AIX_MODULE_PROPERTIES>"]
del deploymentManifest["modulesContent"]["<AIX_MODULE_PROPERTIES>"]
deploymentManifest["modulesContent"][containerImageName] = aiModuleProp

with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

In [ ]:
with open(deploymentManifestFileName) as f:
    deploymentManifest = json.load(f)

lvaModule = deploymentManifest["modulesContent"]["<LVA_MODULE_PROPERTIES>"]
del deploymentManifest["modulesContent"]["<LVA_MODULE_PROPERTIES>"]


for s in lvaSettings:
    lvaModuleS = json.dumps(lvaModule)
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_ADDRESS', s[0])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_USER_NAME', s[1])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_PASSWORD', s[2])
    lvaModuleS = lvaModuleS.replace('AIX_MOTION_SENSITIVITY', s[3])
    lvaModuleS = lvaModuleS.replace('AIX_IP_CAM_FPS_FILTER', s[4])
    lvaModuleS = lvaModuleS.replace('AIX_CONTAINER_SCORING_URI', scoringURI + "?cam=" + str(int(s[0][-3:])))
    temp = json.loads(lvaModuleS)

    deploymentManifest["modulesContent"]["lva" + s[0][-3:]] = temp


with open(deploymentManifestFileName, "w") as f:
    deploymentManifest = json.dump(deploymentManifest, f, indent=4, sort_keys=True)

In [ ]:
# We have a deployment template file which needs to be updated with AI model's container registry address and credentials
file = open(deploymentManifestFileName)
deploymentTemplate = file.read()

if useACRFlag:
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', acrServiceName)
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', acrServiceName+".azurecr.io")
else:
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_NAME', localacrServiceName)
    deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_FULL_NAME', localacrServiceName)

deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_USER_NAME', acrUserName)
deploymentTemplate = deploymentTemplate.replace('AIX_CONTAINER_REGISTRY_PASSWORD', acrPassword)

# Save the resulting deployment file on disk, so we can pass it to deployment API (currently not supporting direct Json string input)
with open(deploymentManifestFileName, 'wt', encoding='utf-8') as outputFile:
    outputFile.write(deploymentTemplate)
    
print(deploymentTemplate)

### 6.2. Deploy to IoT Edge Target
In previous sections, we already set up our physical (or virtual on a VM) IoT Edge Device. Now we connect / match the physical device with the cloud based one on the IoTHub. Everything that we are pushing into IoTHub based edge device will will be cloned into the physical device (all IoT Edge modules, settings etc. defined in the manifest file)

In [ ]:
# Install iot hub CLI extensions. To run below cells, we need iot edge extension
!az extension add --name azure-cli-iot-ext

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Running below cell will reset and remove all previously deployed modules on the edge devices.

In [ ]:
# First Reset the edge device
!az iot edge set-modules --device-id $iotDeviceId --hub-name $iotHubServiceName --content $resetDeviceManifestTemplate

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Warning] </span>  
Running below cell will deploy all modules (our AI extensibility module that we created, lva and all required ones) and settings to the edge devices.

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $iotDeviceId --hub-name $iotHubServiceName --content $deploymentManifestFileName

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  

Once you run the above cell, or make new manifest deployment, run the following shell command on the **IoT Edge Device** to restart the IoT Edge Runtime and the module in order.


```bash
sudo systemctl restart iotedge

sudo iotedge restart <IOT_EDGE_AI_MODULE_NAME>
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  
On the IoT Edge device, run the following shell command to see if all required modules pulled and running. Depending on your Internet connection speed, it may take some time to pull images into IoT Edge device.

```shell
sudo iotedge list
```

Running above command, you should see something similar to below 4 lines, telling that 4 IoTEdge modules (edgeAgent, edgeHub, mediaEdge and mkov01aimodule) pulled, up and running...

```
NAME                        STATUS           DESCRIPTION      CONFIG
edgeAgent                   running          Up 4 minutes     mcr.microsoft.com/azureiotedge-agent:1.0
<IOT_EDGE_AI_MODULE_NAME>   running          Up 4 minutes     mkregistry:55000/mkov01aimodule:latest
<LVA_MODULE_NAME>           running          Up 4 minutes     marketplace.azurecr.io/azure-media-services-preview/official/media_edge:1.2-linux-x64
edgeHub                     running          Up 4 minutes     mcr.microsoft.com/azureiotedge-hub:1.0
```

<span style="color:red; font-weight: bold; font-size:1.1em;"> [!Important] </span>  

Once all modules up and running on the edge device, you should see green icon on the left of "mkov01aimodule" module name in the Azure IoT Hub Extension window in Visual Studio code.

<img src="../doc_imgs/img_03_026.jpg" width=300 alt="> Figure: IoT Edge Module list."/>  

wait 1-2 minute to have IoT Edge Runtime up and running in stable mode. If it is still not green, use the following shell command to restart it:

```shell
sudo iotedge restart <IOT_EDGE_AI_MODULE_NAME>
```

After all, you should see inference results going into logs and IoTHub which are explained in next section, section 9. If you dont observe any message flow or if you think that the pipeline is not working, you should try restarting just the LVA module:

```shell
sudo iotedge restart <LVA_MODULE_NAME>
```